# Dependancy Installations

In [1]:
# transformer library for semantic similrty 
!pip install -U sentence-transformers

     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 2.6 MB 42.7 MB/s 
     |████████████████████████████████| 1.2 MB 25.9 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 636 kB 39.1 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
     |████████████████████████████████| 895 kB 37.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=ebc7eb604c64ebac969d8c6438e8faf840703971408df77f3509e38b0979ff3c
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#  library for approximate nearest neighbor search
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 14 kB/s 


In [3]:

import faiss,numpy

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('stsb-roberta-base-v2')

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/675 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
import spacy
import nltk
import re

In [21]:
import pandas as pd
import numpy as np


# Setting Variables

In [7]:
job_file_path='/content/Jobs.csv'
candidate_file_path='/content/Candidate.csv'

In [8]:
# Relevant columns for job dataframe
job_responsibility_column=8
job_description_column=3
job_qualification_column=9

In [9]:
# Relevant columns for candidate dataframe
candidate_experience_column=5
candidate_education_column=4
candidate_skills_column=9
candidate_certificates_column=11

In [10]:
# Setting Faiss variables
d = 768                   # dimension of vectors
nb = 1638                 # database size (jobs+candidates)
nq = 1                    # nb of queries
k = 1639                  # To see 1000 nearest neighbors

In [11]:
n_jobs=10                 # number of similar jobs to be retrieved
n_candidates=10           # number of similar candidates to be retrieved

# Data Loading

In [13]:
jobs_df=pd.read_csv(job_file_path,encoding = "ISO-8859-1")
cand_df=pd.read_csv(candidate_file_path,encoding = "ISO-8859-1")
del jobs_df['Date']                   # Since contained all NAN values
del jobs_df['Job ID']                 # Since contained all NAN values
jobs_df=jobs_df.dropna()              # Dropping further NAN values

In [14]:
# Concatentaing text from desired column i.e responsibility, description and qualification
job_text=[]
for i in range(len(jobs_df)):
  concatenated_text=(jobs_df.iloc[i,job_responsibility_column]+ '. '+ \
                     jobs_df.iloc[i,job_description_column]+'. '+ \
                     jobs_df.iloc[i,job_qualification_column]).replace('\n','. ')
  text=concatenated_text.replace('\n\n','')
  job_text.append(text)


# Concatentaing text from desired column i.e experience, education,skills and certficates
candidate_text=[]
for i in range(len(cand_df)):
  concatenated_text=str(cand_df.iloc[i,candidate_experience_column])+ '. '+str(cand_df.iloc[i,candidate_education_column])+ \
  '. '+str(cand_df.iloc[i,candidate_skills_column])+'. '+str(cand_df.iloc[i,candidate_certificates_column]).replace('\n','. ')
  text=concatenated_text.replace('\n\n','')
  text=text.replace('\n','')
  candidate_text.append(text)
jobs_df['job_text']=job_text
cand_df['candidate_text']=candidate_text

In [15]:
# Creating set to handle duplicates, since data has lot of duplicates
complete_text=list(set(job_text))+list(set(candidate_text))

# Data Cleaning

In [16]:
# Data preprocessing function
def preprocess(sentence):
  sentence = "".join(filter(lambda x: not x.isdigit(), sentence))
  sentence = ' '.join([i for i in sentence.split() if '@' not in i])
  sentence=re.sub(' +', ' ', sentence)
  sentence=' '.join([item for item in sentence.split() if '@' not in item])
  sentence = ' '.join([item for item in sentence.split() if len(item)>2])
  sentence=re.sub("[\(\[]*?[\)\]]", "", sentence)
  sentence = re.sub(r"[^a-zA-Z0-9.,]+", ' ', sentence)
  sentence=sentence.rstrip("\n")
  sentence=sentence.strip()
  strText=sentence.split()
  sentence=" ".join(strText)
  sentence=sentence.lower()
  return sentence

In [17]:
# Creating list of cleaned text, using above preprocessing function
cleaned_complete=[]
for i in range(len(complete_text)):
  cleaned_complete.append(preprocess(complete_text[i]))

# Generating embeddings and FAISS indexing

In [18]:
# Creating embeddings 
cleaned_embed=model.encode(cleaned_complete,show_progress_bar=True)

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

In [22]:
np.random.seed(1234)             # make reproducible
res = faiss.StandardGpuResources()  # use a single GPU
index_flat = faiss.IndexFlatL2(d)  # build a flat (CPU) index
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
gpu_index_flat.add(cleaned_embed)         # add vectors to the index
print(gpu_index_flat.ntotal)

1639


In [23]:
# Creating distance and index array, here k refers to number of nearest neighbors to be searched
D, I = gpu_index_flat.search(cleaned_embed, k) 

# Extracting similar candidates for a given job

In [25]:
'''
Extracts 10 most similar candidates for a given job along with distance scores
'''

retrieved_candidates=[]
distance=[]
job=[]
for i in range(184):
  act=[]
  dist=[]
  for ele,el in (zip(I[i],D[i])):
    if ele> 184:
      act.append(complete_text[ele])
      dist.append(el)
  job.append(complete_text[i])
  retrieved_candidates.append(act[:10])
  distance.append(dist[:10])

In [27]:
jobs_dataframe=pd.DataFrame(list(zip(job,retrieved_candidates,distance)),columns=['Job Description','Retrieved Candidates','Distance Score'])

In [28]:
# Writing output dataframe as json file
jobs_dataframe.to_json('job_dataframe.json')

# Extracting similar jobs for a given candidate

In [29]:
retrieved_jobs=[]
distance=[]
candidates=[]
for i in range(185,1639):
  act=[]
  dist=[]
  for ele,el in (zip(I[i],D[i])):
    if ele< 185:
      act.append(complete_text[ele])
      dist.append(el)
  candidates.append(complete_text[i])
  retrieved_jobs.append(act[:10])
  distance.append(dist[:10])

In [30]:
candidate_dataframe=pd.DataFrame(list(zip(candidates,retrieved_jobs,distance)),columns=['Candidate Description','Retrieved Jobs','Distance Score'])

In [31]:
# Writing output dataframe as json file
candidate_dataframe.to_json('candidate_dataframe.json')